# Construction des fichiers pour la définition du pipeline de processing textuel

Ce notebook permet d'assembler les fichiers text_process.csv des datasets 1, 3 et 4 en un fichier qui nous servira pour la définition du pipeline de processing textuel


In [1]:
import os
current_dir = %pwd
project_dir = os.path.dirname(current_dir)
%cd $project_dir

c:\Users\Bryan Fernandez\Desktop\DataScientest\doc-classifier


Dans le dossier "data", nous allons créer un dossier "processed" dans lequel nous stockerons les fichiers.

In [2]:
import os
import pandas as pd

if not os.path.exists("data/processed"):
    os.makedirs("data/processed")
    print(f"Folder created: 'data/processed'")
else:
    print(f"Folder already exists: 'data/processed'")

Folder already exists: 'data/processed'


C:\Users\Bryan Fernandez\AppData\Local\Temp\ipykernel_64076\3233540087.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Ce code permet de concaténer les 3 fichiers text_process.csv en ajoutant la colonne 'data' pour avoir le jeu de données d'origine des images.


In [3]:
li = []

for i in ["data_01", "data_03", "data_04"]:
    df = pd.read_csv(f"data/raw/{i}/text_process.csv")
    df['data'] = i
    li.append(df)

frame_words = pd.concat(li, axis=0, ignore_index=True)
frame_words.to_csv("data/processed/words.csv", index=False)

frame_words.head()

,words,lang,lang_score,file_name,category,data
0,center indoor air research winteraon road suit...,en,0.999996,image_0000000,invoice,data_01
1,box kansas city missouri coo claire carcich ph...,en,0.999997,image_0000001,invoice,data_01
2,march attention statement chargo professor ste...,en,0.999996,image_0000002,invoice,data_01
3,ge von ahlendorf iiarat peter ahlendorf gmbh s...,de,0.999996,image_0000003,invoice,data_01
4,send rjr orsyth orsytli tcbaccc gwen scott com...,en,0.999996,image_0000004,invoice,data_01


Ce code permet de concaténer les 3 fichiers text_process.csv ne gardant que les colonnes 'words' et 'category' et en ajoutant la colonne 'count' qui correspond au nombre de mots dans le text.


In [4]:
li = []

for i in ["data_01", "data_03", "data_04"]:
    df = pd.read_csv(f"data/raw/{i}/text_process.csv",
                     usecols=['words', 'category'])
    df['words'] = df['words'].fillna('')
    df.insert(loc=1, column='count', value=df['words'].apply(
        lambda x: len(x.split())))
    li.append(df)

frame_count = pd.concat(li, axis=0, ignore_index=True)
frame_count.to_csv("data/processed/count_words.csv", index=False)

frame_count.head()

,words,count,category
0,center indoor air research winteraon road suit...,27,invoice
1,box kansas city missouri coo claire carcich ph...,43,invoice
2,march attention statement chargo professor ste...,44,invoice
3,ge von ahlendorf iiarat peter ahlendorf gmbh s...,62,invoice
4,send rjr orsyth orsytli tcbaccc gwen scott com...,50,invoice
